In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('Data/updated_ratings.csv')
ratings = ratings.drop('timestamp', axis=1)
ratings = ratings.drop("Unnamed: 0",axis=1)
ratings.head()

ratings

,userId,movieId,rating
0,1,858,5.0
1,1,1246,5.0
2,1,1968,4.0
3,1,2762,4.5
4,1,2959,4.0
...,...,...,...
6935903,270896,30793,3.5
6935904,270896,33166,4.5
6935905,270896,41566,3.5
6935906,270896,48385,4.0


In [3]:
movies= pd.read_csv("Data/merged.csv")
movies = movies[["id", "title"]]
movies.rename(columns = {'id':'movieId'}, inplace = True)
movies.head()

,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [4]:
filter_users = ratings['userId'].value_counts() > ratings['userId'].value_counts().quantile(0.9)
filter_users = filter_users[filter_users].index.tolist()

filter_movies = ratings['movieId'].value_counts() > ratings['movieId'].value_counts().quantile(0.6)
filter_movies = filter_movies[filter_movies].index.tolist()

print(f'Original shape: {ratings.shape}')
ratings = ratings[(ratings['movieId'].isin(filter_movies)) & (ratings['userId'].isin(filter_users))]
print(f'New shape: {ratings.shape}')

Original shape: (6935908, 3)
New shape: (3338177, 3)


Create the rating matrix with rows as movies and columns as users.

In [5]:
data = pd.merge(movies,ratings)
data.head()

,movieId,title,userId,rating
0,862,Toy Story,2103,5.0
1,862,Toy Story,5380,1.0
2,862,Toy Story,6525,4.0
3,862,Toy Story,7050,3.0
4,862,Toy Story,7238,3.0


In [6]:
pivot_table = data.pivot_table(index = ["userId"],columns = ["title"],values = "rating")
pivot_table.head(10)

title,10 Things I Hate About You,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,21 Grams,24 Hour Party People,25th Hour,28 Days Later,28 Weeks Later,2:37,...,Wish You Were Here,Working Girl,X-Men Origins: Wolverine,Yankee Doodle Dandy,You Instead,You Only Live Twice,Young Frankenstein,Young and Dangerous 5,Young and Innocent,eXistenZ
userId,,,,,,,,,,,,,,,,,,,,,
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
24,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN
43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,...,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,2.0,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,5.0,NaN
47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0
49,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
62,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN


In [7]:

def get_recommendation(movie):
    movie_watched = pivot_table[movie]
    similarity_with_other_movies = pivot_table.corrwith(movie_watched)  # find correlation between "Bad Boys (1995)" and other movies
    similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
    print(similarity_with_other_movies.head(25))

movie = "2001: A Space Odyssey"
get_recommendation(movie)

title
2001: A Space Odyssey                 1.000000
Crossing Delancey                     0.573913
China Gate                            0.535783
The Man in Possession                 0.513644
Paranormal Activity: Tokyo Night      0.501438
Crouching Tiger, Hidden Dragon        0.487382
Reservoir Dogs                        0.447803
Tenacious D in The Pick of Destiny    0.446127
Critters                              0.443374
Tarzan and the Amazons                0.440577
Ghost                                 0.438624
War of the Worlds                     0.434168
Lolita                                0.431740
Carry On Cleo                         0.428914
Arsenic and Old Lace                  0.423889
Evil Dead II                          0.422301
My Super Ex-Girlfriend                0.416956
Cape Fear                             0.415947
To Die For                            0.413756
My Son the Fanatic                    0.413731
Snow White and the Seven Dwarfs       0.411968
The Net